<a href="https://colab.research.google.com/github/Cyren4/GCP/blob/main/Gradio_Marketing_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio git+https://github.com/huggingface/transformers gradio torch
!pip install google-cloud-aiplatform --upgrade --user

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai

PROJECT_ID = ""  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.preview.language_models import (ChatModel, InputOutputTextPair,
                                              TextEmbeddingModel,
                                              TextGenerationModel)

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [ ]:
import gradio as gr

# Table 1 : Generate a marketing campaign from brief
def MarketingGenerator(title, content, creativity, channels, locations):
    """Generate a marketing campaign for a given industry."""
    temp = creativity/10

    if not title:
        title = generation_model.predict("Generate maketing campaign title for Gallery Lafayette for " + content, temperature=temp, max_output_tokens=200, top_k=40, top_p=0.8).text
    if "All" in locations :
        locations = ["France"]

    prompt = f"""Generate a Marketing posts for Gallerie Lafayette {content}. The title will be {title} and adapted to {channels}.""" + \
      f""" It will be for all shops in {" and ".join(locations)}. The campaing will be {"super creative" if creativity > 5 else "entertaining"}"""

    result = generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text
    return result

demo1 = gr.Interface(
    MarketingGenerator,
    [
        gr.Textbox(label="Enter your campaign title", info="Leave blank if you don't have one"),
        gr.Textbox(label="Enter campaign brief", info="Core elements"),
        gr.Slider(1, 10, value=4, label="Creativity of your campaign", info="Choose between 1 and 10"),
        gr.Dropdown(
            ["Twitter", "Instagram", "Facebook", "Google Ads", "Website"], label="Pick your communication platforms", info="Adviced to pick one at a time for better result"
        ),
        gr.CheckboxGroup(["Paris", "Lyon", "Besancon", "Marseille", "All"], label="Pick the cities it apply"),
    ],
    outputs=gr.outputs.Textbox(label="Generated campaign"),
    examples=[
        ["Les 3J", "March 2023 sales up to 30% for clothes, shoes, bags.", 10, ["Twitter"], ["Paris", "Marseille"]],
        ["Christmas Special", "Winter 2023 sales up to 40% on all products.", 1, ["Website"], ["All"]],
        ["Father's day", "Get an original gift for father's day among our special selections", 4, ["Facebook"], ["Lyon"]],
    ]
)
# --------------------------------------------------------------------------------------------------------------------- #

# Table 2 : Generate a summary
def summarizer(content):
    prompt = "Provide a summary with about two sentences for the following article: " + content
    result = generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text
    return result

demo2 = gr.Interface(
    summarizer,
    gr.Textbox(label="Enter your full text:"),
    outputs=gr.outputs.Textbox(label="Summarized content"),
    examples=[
        """The fashion industry has a very big impact on how people dress or see themselves. They use advertisements to try to show the public eye how they should. As time goes on the fashion industry is slowly changing. Hannah Berry's article "The Fashion Industry: Free to Be an Individual" addresses how for many years advertisements try to tell women that they have to use certain things or look a certain way to be attractive but now in modern day time the ads are changing to encourage women to be themselves and wear what they feel expresses them. She goes over two shoes ads that both encourage being yourself. Her article reflects the new found light in the fashion industry as now it tries to uplift women an inspire them to have confidence with themselves and to dress the way they want to dress; and look the way the way they want to look. """,
        "The iconic Galeries Lafayette dates from the late nineteenth century, and has become a genuine symbol of the “City of Light”, visited by over twenty-five million people each year. Its magnificently domed central hall – which covers 70,000 square metres (over three quarters of a million square feet) of retail space – fills an entire block of Boulevard Haussmann, one of several Parisian boulevards built by Napoleon. It’s in the 9th arrondissement, and the celebrated novelist Marcel Proust lived just down the road for many years. he raison d’être of Galeries Lafayette has always been to bring innovation and fashion together in the same space, and make them as widely visible and accessible as possible. To this end – and, frankly, to cock a snook at the competition – Théophile Bader took the decision to make high fashion affordable to all budgets, acquiring production capacity to produce his own fashion brand. He produced articles at the leading edge of fashion, but sold them much more affordably than his competitors.Doing this killed two birds with one stone. Access to fashion was democratised: no longer just for the wealthy few; and the store became wildly popular. To this day,efforts continue constantly to diversify its offering, which now includes men’s clothing, interior decoration, toys and tableware. The business works hard to retain as many shoppers as possible, through annual meetings with customers and strategies such as the well-known “3J” sale, created in 1958 to make excellent items of quality available at exceptionally low prices.",
        """In 1969, a new shop for younger clients opened its doors on the other side of rue Mogador. “Club 20” carried cutting-edge fashion, music and drugstore products. In time, departments were added for men’s fashion and “Lafayette Gourmet”. In 1984 the main hall was completely reorganised to contain a variety of smaller boutique stores in the one space.

2001 was a watershed year for the Lafayette brand, with a massive advertising campaign under the banner of “The Adventures of Laetitia Casta in Galeries Lafayette country”. This campaign had very broad reach both to Parisians and to incoming tourists. It was innovative, modern and offbeat, perfectly capturing the values, history and “spirit” of the store through a variety of images. The imaginative use of mannequins pioneered in this campaign continues to this day.

The store also hosts prestigious events with major artists and designers. In order to strengthen and broaden the relationship between Lafayette and fashion, as well as other creative fields, the store opened the “Galerie des Galeries” – a space on the first floor freely accessible to everyone which highlights and explores the links between fashion, art and design.""",
      ]
)

def predict(prompt, max_output_tokens, temperature, top_p, top_k):
    result = generation_model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        top_k=top_k,
        top_p=top_p).text
    return result

demo3 = gr.Interface(
    predict,
    [ gr.Textbox(label="Enter prompt:", value="What are the top 10 trends in the fashion?"),
      gr.Slider(32, 1024, value=512, step = 32, label = "max_output_tokens"),
      gr.Slider(0, 1, value=0.2, step = 0.1, label = "temperature"),
      gr.Slider(0, 1, value=0.8, step = 0.1, label = "top_p"),
      gr.Slider(1, 40, value=38, step = 1, label = "top_k"),
    ],
    outputs=gr.outputs.Textbox(label="Response"),
    examples=[
        ["What are the top 10 trends in the fashion?"],
        ["Generate maketing campaign title for Gallery Lafayette for Winter 2023 sales"],
    ]
)

if __name__ == "__main__":
    demo = gr.TabbedInterface([demo1, demo2, demo3], ["Marketing Generator", "Summarizer", "General text generation"])
    demo.launch(share=True)
